# 说明
这是一篇纯粹的例子。

作用是爬取百度贴吧的某个帖子的图片:
<img src='./image/tieba.png' />

In [1]:
from bs4 import BeautifulSoup
import requests
import contextlib
import sys
import os
import traceback

In [2]:
headers = {'Connection': "keep-alive",
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'zh-CN,zh;q=0.9'}

In [3]:
def Schedule(a, b, c):
    per = 100.0 * a * b / c
    if per > 100:
        per = 100
    print("  " + "%.2f%% 已经下载的大小:%ld 文件大小:%ld" % (per, a * b, c) + '\r')

In [4]:
def urlretrieve(url, filename=None, reporthook=None, params=None):
    print('download file: {0}'.format(url))
    '''传入ID改变url，利用closing跟iter_content下载图片'''
    with contextlib.closing(requests.get(url, stream=True, headers=headers, params=params)) as fp:  # 打开网页
        header = fp.headers
        with open(filename, 'wb+') as tfp:  # w是覆盖原文件，a是追加写入 #打开文件
            bs = 1024
            size = -1
            blocknum = 0
            if "content-length" in header:
                size = int(header["Content-Length"])  # 文件的总大小理论值
            if reporthook and size > 0:
                reporthook(blocknum, bs, size)  # 写入前运行一次回调函数

            for chunk in fp.iter_content(chunk_size=1024):
                if chunk:
                    tfp.write(chunk)  # 写入
                    tfp.flush()
                    blocknum += 1
                    if reporthook and size > 0:
                        reporthook(blocknum, bs, size)  # 每写入一次就运行一次回调函数

In [7]:
def crawl(url):
    try:
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        imglist = soup.find_all('img', class_='BDE_Image')
        folder = './crawlfile/tieba'
        for index, imgtag in enumerate(imglist):
            print('download {0}/{1} files'.format(index+1, len(imglist)))
            imgurl = str(imgtag['src'])
            if imgurl.endswith('.jpg'):
                urlretrieve(url=imgurl,
                            filename=os.path.join(folder,
                                                  '{0}.jpg'.format(index+1)), reporthook=Schedule)


        print(r.encoding)
    except Exception as e:
        print(e)
        traceback.print_exc()

In [9]:
folder = './crawlfile/tieba'
if not os.path.exists(folder):
    os.makedirs(folder)
crawl('http://tieba.baidu.com/p/1753935195')

download 1/67 files
download file: https://imgsa.baidu.com/forum/w%3D580/sign=92f7a739462309f7e76fad1a420f0c39/7eba40ed2e738bd46ea1009ca18b87d6267ff9ae.jpg
  0.00% 已经下载的大小:0 文件大小:10173
  10.07% 已经下载的大小:1024 文件大小:10173
  20.13% 已经下载的大小:2048 文件大小:10173
  30.20% 已经下载的大小:3072 文件大小:10173
  40.26% 已经下载的大小:4096 文件大小:10173
  50.33% 已经下载的大小:5120 文件大小:10173
  60.40% 已经下载的大小:6144 文件大小:10173
  70.46% 已经下载的大小:7168 文件大小:10173
  80.53% 已经下载的大小:8192 文件大小:10173
  90.59% 已经下载的大小:9216 文件大小:10173
  100.00% 已经下载的大小:10240 文件大小:10173
download 2/67 files
download file: https://imgsa.baidu.com/forum/w%3D580/sign=71ef1fbf720e0cf3a0f74ef33a47f23d/17128d01a18b87d6c6c219b2070828381e30fdae.jpg
  0.00% 已经下载的大小:0 文件大小:10577
  9.68% 已经下载的大小:1024 文件大小:10577
  19.36% 已经下载的大小:2048 文件大小:10577
  29.04% 已经下载的大小:3072 文件大小:10577
  38.73% 已经下载的大小:4096 文件大小:10577
  48.41% 已经下载的大小:5120 文件大小:10577
  58.09% 已经下载的大小:6144 文件大小:10577
  67.77% 已经下载的大小:7168 文件大小:10577
  77.45% 已经下载的大小:8192 文件大小:10577
  87.13% 已经下载的大小:9216 文件大小:10577
  

  0.00% 已经下载的大小:0 文件大小:12763
  8.02% 已经下载的大小:1024 文件大小:12763
  16.05% 已经下载的大小:2048 文件大小:12763
  24.07% 已经下载的大小:3072 文件大小:12763
  32.09% 已经下载的大小:4096 文件大小:12763
  40.12% 已经下载的大小:5120 文件大小:12763
  48.14% 已经下载的大小:6144 文件大小:12763
  56.16% 已经下载的大小:7168 文件大小:12763
  64.19% 已经下载的大小:8192 文件大小:12763
  72.21% 已经下载的大小:9216 文件大小:12763
  80.23% 已经下载的大小:10240 文件大小:12763
  88.26% 已经下载的大小:11264 文件大小:12763
  96.28% 已经下载的大小:12288 文件大小:12763
  100.00% 已经下载的大小:13312 文件大小:12763
download 13/67 files
download file: https://imgsa.baidu.com/forum/w%3D580/sign=34d2f1ac0a55b3199cf9827d73a88286/364bdc88d43f87943fd3f583d21b0ef41bd53a79.jpg
  0.00% 已经下载的大小:0 文件大小:10216
  10.02% 已经下载的大小:1024 文件大小:10216
  20.05% 已经下载的大小:2048 文件大小:10216
  30.07% 已经下载的大小:3072 文件大小:10216
  40.09% 已经下载的大小:4096 文件大小:10216
  50.12% 已经下载的大小:5120 文件大小:10216
  60.14% 已经下载的大小:6144 文件大小:10216
  70.16% 已经下载的大小:7168 文件大小:10216
  80.19% 已经下载的大小:8192 文件大小:10216
  90.21% 已经下载的大小:9216 文件大小:10216
  100.00% 已经下载的大小:10240 文件大小:10216
download 14/67 files

  0.00% 已经下载的大小:0 文件大小:41408
  2.47% 已经下载的大小:1024 文件大小:41408
  4.95% 已经下载的大小:2048 文件大小:41408
  7.42% 已经下载的大小:3072 文件大小:41408
  9.89% 已经下载的大小:4096 文件大小:41408
  12.36% 已经下载的大小:5120 文件大小:41408
  14.84% 已经下载的大小:6144 文件大小:41408
  17.31% 已经下载的大小:7168 文件大小:41408
  19.78% 已经下载的大小:8192 文件大小:41408
  22.26% 已经下载的大小:9216 文件大小:41408
  24.73% 已经下载的大小:10240 文件大小:41408
  27.20% 已经下载的大小:11264 文件大小:41408
  29.68% 已经下载的大小:12288 文件大小:41408
  32.15% 已经下载的大小:13312 文件大小:41408
  34.62% 已经下载的大小:14336 文件大小:41408
  37.09% 已经下载的大小:15360 文件大小:41408
  39.57% 已经下载的大小:16384 文件大小:41408
  42.04% 已经下载的大小:17408 文件大小:41408
  44.51% 已经下载的大小:18432 文件大小:41408
  46.99% 已经下载的大小:19456 文件大小:41408
  49.46% 已经下载的大小:20480 文件大小:41408
  51.93% 已经下载的大小:21504 文件大小:41408
  54.40% 已经下载的大小:22528 文件大小:41408
  56.88% 已经下载的大小:23552 文件大小:41408
  59.35% 已经下载的大小:24576 文件大小:41408
  61.82% 已经下载的大小:25600 文件大小:41408
  64.30% 已经下载的大小:26624 文件大小:41408
  66.77% 已经下载的大小:27648 文件大小:41408
  69.24% 已经下载的大小:28672 文件大小:41408
  71.72% 已经下载的大小:29696 文件大小:4140

  0.00% 已经下载的大小:0 文件大小:13201
  7.76% 已经下载的大小:1024 文件大小:13201
  15.51% 已经下载的大小:2048 文件大小:13201
  23.27% 已经下载的大小:3072 文件大小:13201
  31.03% 已经下载的大小:4096 文件大小:13201
  38.78% 已经下载的大小:5120 文件大小:13201
  46.54% 已经下载的大小:6144 文件大小:13201
  54.30% 已经下载的大小:7168 文件大小:13201
  62.06% 已经下载的大小:8192 文件大小:13201
  69.81% 已经下载的大小:9216 文件大小:13201
  77.57% 已经下载的大小:10240 文件大小:13201
  85.33% 已经下载的大小:11264 文件大小:13201
  93.08% 已经下载的大小:12288 文件大小:13201
  100.00% 已经下载的大小:13312 文件大小:13201
download 24/67 files
download file: https://imgsa.baidu.com/forum/w%3D580%3Bcp%3Dtieba%2C10%2C392%3Bap%3D%CB%EF%C7%ED%B0%C9%2C90%2C400/sign=13a537d58b82b9013dadc33b43b6ca07/2685ac1ea8d3fd1f9d6dbec8314e251f94ca5fee.jpg
  0.00% 已经下载的大小:0 文件大小:86177
  1.19% 已经下载的大小:1024 文件大小:86177
  2.38% 已经下载的大小:2048 文件大小:86177
  3.56% 已经下载的大小:3072 文件大小:86177
  4.75% 已经下载的大小:4096 文件大小:86177
  5.94% 已经下载的大小:5120 文件大小:86177
  7.13% 已经下载的大小:6144 文件大小:86177
  8.32% 已经下载的大小:7168 文件大小:86177
  9.51% 已经下载的大小:8192 文件大小:86177
  10.69% 已经下载的大小:9216 文件大小:86177
  

  0.00% 已经下载的大小:0 文件大小:36010
  2.84% 已经下载的大小:1024 文件大小:36010
  5.69% 已经下载的大小:2048 文件大小:36010
  8.53% 已经下载的大小:3072 文件大小:36010
  11.37% 已经下载的大小:4096 文件大小:36010
  14.22% 已经下载的大小:5120 文件大小:36010
  17.06% 已经下载的大小:6144 文件大小:36010
  19.91% 已经下载的大小:7168 文件大小:36010
  22.75% 已经下载的大小:8192 文件大小:36010
  25.59% 已经下载的大小:9216 文件大小:36010
  28.44% 已经下载的大小:10240 文件大小:36010
  31.28% 已经下载的大小:11264 文件大小:36010
  34.12% 已经下载的大小:12288 文件大小:36010
  36.97% 已经下载的大小:13312 文件大小:36010
  39.81% 已经下载的大小:14336 文件大小:36010
  42.65% 已经下载的大小:15360 文件大小:36010
  45.50% 已经下载的大小:16384 文件大小:36010
  48.34% 已经下载的大小:17408 文件大小:36010
  51.19% 已经下载的大小:18432 文件大小:36010
  54.03% 已经下载的大小:19456 文件大小:36010
  56.87% 已经下载的大小:20480 文件大小:36010
  59.72% 已经下载的大小:21504 文件大小:36010
  62.56% 已经下载的大小:22528 文件大小:36010
  65.40% 已经下载的大小:23552 文件大小:36010
  68.25% 已经下载的大小:24576 文件大小:36010
  71.09% 已经下载的大小:25600 文件大小:36010
  73.94% 已经下载的大小:26624 文件大小:36010
  76.78% 已经下载的大小:27648 文件大小:36010
  79.62% 已经下载的大小:28672 文件大小:36010
  82.47% 已经下载的大小:29696 文件大小:360

...

  0.00% 已经下载的大小:0 文件大小:39872
  2.57% 已经下载的大小:1024 文件大小:39872
  5.14% 已经下载的大小:2048 文件大小:39872
  7.70% 已经下载的大小:3072 文件大小:39872
  10.27% 已经下载的大小:4096 文件大小:39872
  12.84% 已经下载的大小:5120 文件大小:39872
  15.41% 已经下载的大小:6144 文件大小:39872
  17.98% 已经下载的大小:7168 文件大小:39872
  20.55% 已经下载的大小:8192 文件大小:39872
  23.11% 已经下载的大小:9216 文件大小:39872
  25.68% 已经下载的大小:10240 文件大小:39872
  28.25% 已经下载的大小:11264 文件大小:39872
  30.82% 已经下载的大小:12288 文件大小:39872
  33.39% 已经下载的大小:13312 文件大小:39872
  35.96% 已经下载的大小:14336 文件大小:39872
  38.52% 已经下载的大小:15360 文件大小:39872
  41.09% 已经下载的大小:16384 文件大小:39872
  43.66% 已经下载的大小:17408 文件大小:39872
  46.23% 已经下载的大小:18432 文件大小:39872
  48.80% 已经下载的大小:19456 文件大小:39872
  51.36% 已经下载的大小:20480 文件大小:39872
  53.93% 已经下载的大小:21504 文件大小:39872
  56.50% 已经下载的大小:22528 文件大小:39872
  59.07% 已经下载的大小:23552 文件大小:39872
  61.64% 已经下载的大小:24576 文件大小:39872
  64.21% 已经下载的大小:25600 文件大小:39872
  66.77% 已经下载的大小:26624 文件大小:39872
  69.34% 已经下载的大小:27648 文件大小:39872
  71.91% 已经下载的大小:28672 文件大小:39872
  74.48% 已经下载的大小:29696 文件大小:398

  0.00% 已经下载的大小:0 文件大小:53894
  1.90% 已经下载的大小:1024 文件大小:53894
  3.80% 已经下载的大小:2048 文件大小:53894
  5.70% 已经下载的大小:3072 文件大小:53894
  7.60% 已经下载的大小:4096 文件大小:53894
  9.50% 已经下载的大小:5120 文件大小:53894
  11.40% 已经下载的大小:6144 文件大小:53894
  13.30% 已经下载的大小:7168 文件大小:53894
  15.20% 已经下载的大小:8192 文件大小:53894
  17.10% 已经下载的大小:9216 文件大小:53894
  19.00% 已经下载的大小:10240 文件大小:53894
  20.90% 已经下载的大小:11264 文件大小:53894
  22.80% 已经下载的大小:12288 文件大小:53894
  24.70% 已经下载的大小:13312 文件大小:53894
  26.60% 已经下载的大小:14336 文件大小:53894
  28.50% 已经下载的大小:15360 文件大小:53894
  30.40% 已经下载的大小:16384 文件大小:53894
  32.30% 已经下载的大小:17408 文件大小:53894
  34.20% 已经下载的大小:18432 文件大小:53894
  36.10% 已经下载的大小:19456 文件大小:53894
  38.00% 已经下载的大小:20480 文件大小:53894
  39.90% 已经下载的大小:21504 文件大小:53894
  41.80% 已经下载的大小:22528 文件大小:53894
  43.70% 已经下载的大小:23552 文件大小:53894
  45.60% 已经下载的大小:24576 文件大小:53894
  47.50% 已经下载的大小:25600 文件大小:53894
  49.40% 已经下载的大小:26624 文件大小:53894
  51.30% 已经下载的大小:27648 文件大小:53894
  53.20% 已经下载的大小:28672 文件大小:53894
  55.10% 已经下载的大小:29696 文件大小:53894

  3.62% 已经下载的大小:1024 文件大小:28253
  7.25% 已经下载的大小:2048 文件大小:28253
  10.87% 已经下载的大小:3072 文件大小:28253
  14.50% 已经下载的大小:4096 文件大小:28253
  18.12% 已经下载的大小:5120 文件大小:28253
  21.75% 已经下载的大小:6144 文件大小:28253
  25.37% 已经下载的大小:7168 文件大小:28253
  29.00% 已经下载的大小:8192 文件大小:28253
  32.62% 已经下载的大小:9216 文件大小:28253
  36.24% 已经下载的大小:10240 文件大小:28253
  39.87% 已经下载的大小:11264 文件大小:28253
  43.49% 已经下载的大小:12288 文件大小:28253
  47.12% 已经下载的大小:13312 文件大小:28253
  50.74% 已经下载的大小:14336 文件大小:28253
  54.37% 已经下载的大小:15360 文件大小:28253
  57.99% 已经下载的大小:16384 文件大小:28253
  61.61% 已经下载的大小:17408 文件大小:28253
  65.24% 已经下载的大小:18432 文件大小:28253
  68.86% 已经下载的大小:19456 文件大小:28253
  72.49% 已经下载的大小:20480 文件大小:28253
  76.11% 已经下载的大小:21504 文件大小:28253
  79.74% 已经下载的大小:22528 文件大小:28253
  83.36% 已经下载的大小:23552 文件大小:28253
  86.99% 已经下载的大小:24576 文件大小:28253
  90.61% 已经下载的大小:25600 文件大小:28253
  94.23% 已经下载的大小:26624 文件大小:28253
  97.86% 已经下载的大小:27648 文件大小:28253
  100.00% 已经下载的大小:28672 文件大小:28253
download 50/67 files
download file: https://imgsa.baidu.co

  0.00% 已经下载的大小:0 文件大小:12846
  7.97% 已经下载的大小:1024 文件大小:12846
  15.94% 已经下载的大小:2048 文件大小:12846
  23.91% 已经下载的大小:3072 文件大小:12846
  31.89% 已经下载的大小:4096 文件大小:12846
  39.86% 已经下载的大小:5120 文件大小:12846
  47.83% 已经下载的大小:6144 文件大小:12846
  55.80% 已经下载的大小:7168 文件大小:12846
  63.77% 已经下载的大小:8192 文件大小:12846
  71.74% 已经下载的大小:9216 文件大小:12846
  79.71% 已经下载的大小:10240 文件大小:12846
  87.68% 已经下载的大小:11264 文件大小:12846
  95.66% 已经下载的大小:12288 文件大小:12846
  100.00% 已经下载的大小:13312 文件大小:12846
download 56/67 files
download file: https://imgsa.baidu.com/forum/w%3D580%3Bcp%3Dtieba%2C10%2C483%3Bap%3D%CB%EF%C7%ED%B0%C9%2C90%2C491/sign=af793617ac6eddc426e7b4f309e0d58d/d8d7871001e9390180b295cc7aec54e737d1969a.jpg
  0.00% 已经下载的大小:0 文件大小:43180
  2.37% 已经下载的大小:1024 文件大小:43180
  4.74% 已经下载的大小:2048 文件大小:43180
  7.11% 已经下载的大小:3072 文件大小:43180
  9.49% 已经下载的大小:4096 文件大小:43180
  11.86% 已经下载的大小:5120 文件大小:43180
  14.23% 已经下载的大小:6144 文件大小:43180
  16.60% 已经下载的大小:7168 文件大小:43180
  18.97% 已经下载的大小:8192 文件大小:43180
  21.34% 已经下载的大小:9216 文件大小:4318

  0.00% 已经下载的大小:0 文件大小:19818
  5.17% 已经下载的大小:1024 文件大小:19818
  10.33% 已经下载的大小:2048 文件大小:19818
  15.50% 已经下载的大小:3072 文件大小:19818
  20.67% 已经下载的大小:4096 文件大小:19818
  25.84% 已经下载的大小:5120 文件大小:19818
  31.00% 已经下载的大小:6144 文件大小:19818
  36.17% 已经下载的大小:7168 文件大小:19818
  41.34% 已经下载的大小:8192 文件大小:19818
  46.50% 已经下载的大小:9216 文件大小:19818
  51.67% 已经下载的大小:10240 文件大小:19818
  56.84% 已经下载的大小:11264 文件大小:19818
  62.00% 已经下载的大小:12288 文件大小:19818
  67.17% 已经下载的大小:13312 文件大小:19818
  72.34% 已经下载的大小:14336 文件大小:19818
  77.51% 已经下载的大小:15360 文件大小:19818
  82.67% 已经下载的大小:16384 文件大小:19818
  87.84% 已经下载的大小:17408 文件大小:19818
  93.01% 已经下载的大小:18432 文件大小:19818
  98.17% 已经下载的大小:19456 文件大小:19818
  100.00% 已经下载的大小:20480 文件大小:19818
download 61/67 files
download file: https://imgsa.baidu.com/forum/w%3D580%3Bcp%3Dtieba%2C10%2C483%3Bap%3D%CB%EF%C7%ED%B0%C9%2C90%2C491/sign=fe9088dca9d3fd1f3609a2320075466f/a73551b5c9ea15ce4ac8fbfbb7003af33a87b228.jpg
  0.00% 已经下载的大小:0 文件大小:13597
  7.53% 已经下载的大小:1024 文件大小:13597
  15.06% 已经下载的大小:2048